In [ ]:
import os
import requests
import tempfile
import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client
from moviepy.editor import VideoFileClip, AudioFileClip

In [ ]:
load_dotenv()

CONFIG VARIABLES

In [ ]:
session_id = 14
interview_id = 31
bucket_name = 'interviews'

supabase_client = create_client('https://kglmfklezrjwfvtcolgb.supabase.co', os.environ.get('SUPABASE_KEY'))
supabase: Client = create_client('https://kglmfklezrjwfvtcolgb.supabase.co', os.environ.get('SUPABASE_KEY'))
supabase_connection = supabase_client.storage.from_('interviews')

DIARIZATION_API_URL = 'https://transcribe.whisperapi.com'
STT_API_URL = 'https://api.lemonfox.ai/v1/audio/transcriptions'

headers = {
  'Authorization': 'Bearer {}'.format(os.environ.get('WHISPER_API_KEY'))
}

Current method save_results_to_bd

In [ ]:
def save_results_to_bd(results: pd.DataFrame) -> None:
    try:
        response = supabase.table('interviews').select('user_id').eq('id', interview_id).execute()
        user_id = response.data[0]['user_id']

        results['interview_id'] = interview_id
        results['user_id'] = user_id
        results = results.fillna('')

        data_to_insert = results.to_dict(orient='records')

        response = supabase.table('results').insert(data_to_insert).execute()
        print('{} lines saved to the database successfully'. format(len(response.data)))
    except Exception as e:
        print('Error saving results to the database', str(e))

Current method save df to bd

In [ ]:
def save_results_to_bd(results: pd.DataFrame) -> None:
    try:
        response = supabase.table('interviews').select('user_id').eq('id', interview_id).execute()
        user_id = response.data[0]['user_id']

        results['interview_id'] = interview_id
        results['user_id'] = user_id
        results = results.fillna('')

        data_to_insert = results.to_dict(orient='records')

        response = supabase.table('results').insert(data_to_insert).execute()
        print('{} lines saved to the database successfully'. format(len(response.data)))
    except Exception as e:
        print('Error saving results to the database', str(e))

Current method open_input_file (to modify)

In [ ]:
def video_to_audio(video_name: str):
    s3_path = '{}/{}/raw/{}'.format(session_id, interview_id, video_name)
    
    video_bytes = supabase_connection.download(s3_path)
    with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as video_file:
        video_path = video_file.name
        try:
            video_file.write(video_bytes)
            video_file.flush()
            video_file.close()
            
            video_clip = VideoFileClip(video_path)
            audio = video_clip.audio
            
            with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as audio_file:
                audio_path = audio_file.name
                try:
                    audio.write_audiofile(audio_path, codec='pcm_s16le')
                except Exception as e:
                    raise e
                finally:
                    audio_file.close()
        finally:
            video_clip.close()
            os.remove(video_path)
    return audio_path

In [ ]:
audio_path = video_to_audio('raw.mp4')

In [ ]:
def diarize(audio_path: str) -> pd.DataFrame:
    file = {'file': open(audio_path, 'rb')}
    data = {'fileType': 'wav',
            'diarization': 'true',
            'numSpeakers': '2',
            'language': 'fr',
            'task': 'transcribe'
            }
    response = requests.post(DIARIZATION_API_URL, headers=headers, data=data, files=file)
    df = pd.DataFrame(response.json()['diarization'])
    df.rename(columns={'startTime': 'start', 'stopTime': 'end'}, inplace=True)
    return df

In [ ]:
def speech_to_text(audio_path: str, diarization: pd.DataFrame) -> pd.DataFrame:
    
    audio = AudioFileClip(audio_path)
    results = pd.DataFrame(columns=['start', 'end', 'speaker', 'interview_id', 'user_id', 'text', 'filepath']) 
    
    for i, row in diarization.iterrows():
        audio_segment = audio.subclip(row.start, row.end)
        
        with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as audio_clip_file:
            audio_clip_path = audio_clip_file.name
            try:
                audio_segment.write_audiofile(audio_clip_path, codec='pcm_s16le')
                
                
                data = {'language': 'fr',
                        'response_format': 'text'
                    }
                file = {'file': open(audio_clip_path, 'rb')}
                
                response = requests.post(STT_API_URL, headers=headers, files=file, data=data)
                if response.status_code == 200:
                    results = pd.concat([results, pd.DataFrame([row.to_dict()])], ignore_index=True)
                    idx = results.index[-1]
                    results.at[idx, 'filepath'] = audio_clip_path
                    results.at[idx, 'text'] = response.json()
                    
            except Exception as e:
                raise e
            finally:
                audio_clip_file.close()
                
    return results

In [ ]:
df = diarize(audio_path)

df_full = speech_to_text(audio_path, df)
df_full['speaker'] = df_full['speaker'].map(lambda x: int(x.split('_')[1]))

In [ ]:
df_final = df_full.drop(columns=['filepath'])
df_final['start'] = df_final['start'] * 1000
df_final['end'] = df_final['end'] * 1000
save_results_to_bd(df_final)